In [3]:
import random
import pandas as pd
from sage.graphs.graph_generators import graphs  # Uvoz SageMath grafov


grafi = []  

def cikel(n):
    G = Graph()
    G.add_vertices(range(n))
    e = [(i, (i + 1) % n) for i in range(n)]
    G.add_edges(e)
    return G

def drevo(i):
    return [Graph(tree) for tree in graphs.trees(i)]

 
for i in range(3, 13):  
    grafi.append(cikel(i))
    grafi.extend(drevo(i))


# 2. možnost generiranja grafov - več časa in prostora potrebuje za generiranje grafov:    
# Zanka za generiranje grafov za število vozlišč od 1 do 8 (velja, da je zgornja meja (n+1), kjer je n največe število vozlišč, ki jih lahko ima graf) 
# lahko bi izbrali tudi večji n, vendar se mi v CoClacu program ustavi, saj je mu zmanjka prostora za shranjevanje

# for i in range(1, 9):
#    for G in graphs.nauty_geng(f'{i} -c'):  # 'c' pomeni povezane grafe
#        # Dodaj graf G v seznam grafov
#        grafi.append(G)


###########################################################

def SDCTD_stevilo2(g):
    h = g.complement()

    p = MixedIntegerLinearProgram(maximization = False)
    x = p.new_variable(binary = True)
    p.set_objective(sum([x[v] for v in g]))

    for v in g.vertices():
        neighbors = g.neighbors(v)  
        if neighbors:
            p.add_constraint(sum((x[w] + x[v]) for w in neighbors) >= 1)
        else:
            p.add_constraint(x[v] == 1)
            
    zaustavi = False

    for v in g.vertices():
        neighbors_complement = h.neighbors(v)  
        if neighbors_complement:  
            p.add_constraint(sum(x[w] for w in neighbors_complement) >= 1)

        else: 
            zaustavi = True
            break 

    if not zaustavi == True:
        p.solve()
        x = p.get_values(x)
        return(sum(1 for i in x.values() if i == 1))
    else:
        return None
    
############################################################ 

rezultati = []

# izberemo naključno grafe iz množice grafi in določimo kartezični produkt --- za vse tri grafe določimo SDCTD
# lahko bi izbrali tudi večji n oz. m, vendar se mi v CoClacu program ustavi, saj je mu zmanjka prostora za shranjevanje

# v članku sem zasledil, da je za dokazovanje Vizingove domeve najbolj fiksirati en graf, drugega pa naključno izbiramo
# zato sem fiksira g1 in za njega poiskal kartezični produkt z desetimi različnimi g2, nato sem to večkrat ponovil

for n in range(1, 50):
    g1 = random.choice(grafi)   

    for m in range(1, 10):
        g2 = random.choice(grafi)
        # Kartezični produkt med g1 in g2
        g3 = g1.cartesian_product(g2)

        # Izračun dominacijskega števila za vsak graf
        dominacijsko_stevilo_graf1 = SDCTD_stevilo2(g1) 
        dominacijsko_stevilo_graf2 = SDCTD_stevilo2(g2)
        dominacijsko_stevilo_graf3 = SDCTD_stevilo2(g3)

        # Dodajanje rezultatov v seznam
        rezultati.append({
            'Izbira grafov': f'Izbira {n}',
            'SDCTD_graf1': dominacijsko_stevilo_graf1,
            'SDCTD_graf2': dominacijsko_stevilo_graf2,
            'SDCTD_kartezicni': dominacijsko_stevilo_graf3
        })

    pd.set_option('display.max_rows', None)
    # Ustvari pandas DataFrame iz seznama rezultatov
    df = pd.DataFrame(rezultati)


    def vizingova_domneva(row):
        # Preveri, ali je katera od vrednosti None
        if row['SDCTD_graf1'] is None or row['SDCTD_graf2'] is None:
            return True
        # Preveri, če velja pogoj za produkt in kartezično vrednost
        if row['SDCTD_graf1'] * row['SDCTD_graf2'] <= row['SDCTD_kartezicni']:
            return True
        # V vseh drugih primerih
        return False


# Dodaj stolpec 'Vizingova domneva'
df['Vizingova domneva'] = df.apply(vizingova_domneva, axis=1)

# Izpiši DataFrame
print(df)

    Izbira grafov SDCTD_graf1 SDCTD_graf2 SDCTD_kartezicni  Vizingova domneva
0        Izbira 1           3           3               26               True
1        Izbira 1           3           3               28               True
2        Izbira 1           3           4               31               True
3        Izbira 1           3           3               23               True
4        Izbira 1           3        None               12               True
5        Izbira 1           3           3               25               True
6        Izbira 1           3           3               28               True
7        Izbira 1           3           4               29               True
8        Izbira 1           3           4               31               True
9        Izbira 2           4           3               26               True
10       Izbira 2           4           3               21               True
11       Izbira 2           4           3               18      